In [1]:
import torch
print("Broj GPU-ova:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"{i}: {torch.cuda.get_device_name(i)}")


Broj GPU-ova: 4
0: Tesla V100-SXM2-16GB
1: Tesla V100-SXM2-16GB
2: Tesla V100-SXM2-16GB
3: Tesla V100-SXM2-16GB


# Model twitter-roberta-base-sentiment

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer
from datasets import load_from_disk
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

2025-10-28 00:21:50.262220: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 00:21:50.322253: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-28 00:21:51.934250: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [4]:
train_dataset = load_from_disk("tokenized/train")
val_dataset = load_from_disk("tokenized/val")
test_dataset = load_from_disk("tokenized/test")

train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  
    ignore_mismatched_sizes=True
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    }


In [7]:
training_args = TrainingArguments(
    output_dir="./twitter-roberta-base-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs3",
    logging_steps=100
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipykernel_29514/2116854680.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.134300,0.126902,0.952980,0.952980,0.952984,0.952980
2,0.118100,0.121968,0.956014,0.956013,0.956031,0.956014
3,0.105600,0.121301,0.956905,0.956905,0.956908,0.956905


/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [9]:
metrics = trainer.evaluate(test_dataset)
print(metrics)


/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.12324731051921844, 'eval_accuracy': 0.9565248913122283, 'eval_f1': 0.9565248540130835, 'eval_precision': 0.956526453580769, 'eval_recall': 0.9565248913122283, 'eval_runtime': 255.1355, 'eval_samples_per_second': 1567.79, 'eval_steps_per_second': 3.065, 'epoch': 5.0}


In [10]:
trainer.save_model("./twitter-roberta-base-sentiment")
tokenizer.save_pretrained("./twitter-roberta-base-sentiment")

('./twitter-roberta-base-sentiment/tokenizer_config.json',
 './twitter-roberta-base-sentiment/special_tokens_map.json',
 './twitter-roberta-base-sentiment/vocab.json',
 './twitter-roberta-base-sentiment/merges.txt',
 './twitter-roberta-base-sentiment/added_tokens.json')